In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from rdkit import Chem
from dfs_transformer.utils import DFSCode2Graph, Graph2Mol, isValid, Smiles2DFSCode, DFSCode2Smiles, isValidMoleculeDFSCode
from dfs_transformer.utils import load_selfattn_wandb, load_selfattn_local
import os.path
import torch
import torch.nn as nn
import dfs_transformer as dfs
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 34 files to the new cache system


  0%|          | 0/34 [00:00<?, ?it/s]

In [2]:
pretrained_model = "r2r-nofeat"
if os.path.isdir("../../wandb/artifacts/%s"%pretrained_model):
    bert, cfg = load_selfattn_local("../../wandb/artifacts/%s"%pretrained_model)
else:
    bert, cfg = load_selfattn_wandb(pretrained_model, wandb_dir="../../wandb")

2022/10/28 18:22:31	ERROR	wandb.jupyter	Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eth-compiler-opters (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Downloading large artifact r2r-nofeat:latest, 98.80MB. 2 files... Done. 0:0:0


In [3]:
bert.eval()

DFSCodeSeq2SeqFC(
  (encoder): DFSCodeEncoder(
    (emb_dfs): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (emb_seq): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (emb_atom): Linear(in_features=118, out_features=120, bias=True)
    (emb_bond): Linear(in_features=5, out_features=120, bias=True)
    (mixer): Linear(in_features=600, out_features=600, bias=True)
    (enc): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=600, out_features=600, bias=True)
          )
          (linear1): Linear(in_features=600, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=600, bias=True)
          (norm1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((600,), eps=1e-05, e

In [4]:
smiles_list = 1*['CN=C=O'] + 5*['Cc1cn2c(CN(C)C(=O)c3ccc(F)cc3C)c(C)nc2s1'] + 5*['Cc1cc(F)ccc1C(=O)N(C)Cc1c(C)nc2scc(C)n12']

In [5]:
dlist = []
for smiles in smiles_list:
    data = dfs.smiles2graph(smiles)
    data.edge_features = data.edge_attr
    data.node_features = nn.functional.one_hot(data.z-1, num_classes=118).float()
    dlist += [data]

/home/chrisw/Documents/projects/2023/graph-transformer/notebooks/generative_modelling/../../src/dfs_transformer/datasets/utils.py:300: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  x = torch.tensor([atomic_number, aromatic, sp, sp2, sp3, num_hs],


In [6]:
inputs, outputs = dfs.collate_BERT(dlist, mode="rnd2rnd", fraction_missing=0.15)

IndexError: argmax(): Expected reduction dim 2 to have non-zero size.

In [ ]:
code_list = bert.fwd_code(inputs)
valid_list = []
mols = []
for smiles, code in zip(smiles_list, code_list):
    try:
        valid_list += [isValidMoleculeDFSCode(code)]
        if valid_list[-1]:
            print('input :', smiles)
            print('output:', DFSCode2Smiles(code))
            mols += [Chem.MolFromSmiles(smiles)]
            mols += [Chem.MolFromSmiles(DFSCode2Smiles(code))]
    except:
        valid_list += [False]
valid = np.asarray(valid_list)
print(valid.sum()/len(valid))
Chem.Draw.MolsToGridImage(mols, molsPerRow=2)

In [ ]:
code_list = bert.fwd_code_all(inputs)
valid_list = []
mols = []
for smiles, code in zip(smiles_list, code_list):
    try:
        valid_list += [isValidMoleculeDFSCode(code)]
        if valid_list[-1]:
            print('input :', smiles)
            print('output:', DFSCode2Smiles(code))
            mols += [Chem.MolFromSmiles(smiles)]
            mols += [Chem.MolFromSmiles(DFSCode2Smiles(code))]
    except:
        valid_list += [False]
valid = np.asarray(valid_list)
Chem.Draw.MolsToGridImage(mols, molsPerRow=2)

In [ ]:
inputs['atm_from'].shape

In [ ]:
inputs = {'dfs_from': torch.ones(25, 10, dtype=torch.long)*(-1),
          'dfs_to': torch.ones(25, 10, dtype=torch.long)*(-1),
          'atm_from': torch.ones(25, 10, 118)*(-1),
          'atm_to':torch.ones(25, 10, 118)*(-1),
          'bnd':torch.ones(25, 10, 5)*(-1)}

In [ ]:
inputs['dfs_from'][0] = 0
inputs['dfs_to'][0] = 1

In [ ]:
bert.fwd_code_all(inputs)

In [ ]:
inputs